# <font color="#33ccff">Notebook for downloading and scoring tweets that match given hashtags</font>
<img src=https://www.softwareadvice.com/resources/wp-content/uploads/The-Best-Free-Tools-for-Twitter-Sentiment-Analysis-Tile.png>

### Set credentials

In [185]:
consumer_key = "consumer_key"
consumer_secret = "consumer_secret"
access_token_key = "access_token_key"
access_token_secret = "asccess_token_secret"

credentials = dict(
    consumer_key=consumer_key,
    consumer_secret=consumer_secret,
    access_token_key=access_token_key,
    access_token_secret=access_token_secret
)

### Save credentials to file

In [186]:
# import pickle

# filename = "credentials"
# with open(filename, "wb") as cred_fd:
#     pickle.dump(credentials, cred_fd)

### Load credentials 
<font color='red'>NOTE:</font> that you don't want other people to know them, so do not commit and push accidently do github or something

In [187]:
import pickle

with open("credentials", "rb") as cred_fd:
    credentials = pickle.load(cred_fd)

In [188]:
import logging
import os

logging.basicConfig(level=logging.INFO)


In [189]:
import twitter


class TwitterDownloader:
    def __init__(self, credentials):
        self.api = twitter.Api(**credentials, sleep_on_rate_limit=True)
        self.data = []
      
    @staticmethod
    def extract_tweet_data(tweet):
        return {
            "created_at" : tweet.created_at,
            "created_at_seconds" : tweet.created_at_in_seconds,
            "id" : tweet.id_str,
            "lang" : tweet.lang,
            "user" : tweet.user.name,
            "full_text" : tweet.full_text
        }
    
    def tweets_with_hashtags_generator(self, hashtags, lang, max_tweets=1000):
        """
        Download tweets with given hashtags. Minimum is 100.
        :param hashtags: list of lists, if you want tweets with multiple hashtags inside or string (or 1 elem list) if 
                         only one hashtag in tweet.
        """
        downloaded_tweets = 0
        previous_downloaded = -1
        max_id = 0
        max_ids = dict((el,0) for el in hashtags) 
        while downloaded_tweets < max_tweets:
            if previous_downloaded == downloaded_tweets:
                print("No more tweets to download, ending...")
                break
                
                
            previous_downloaded = downloaded_tweets
            for in_tweet_hashtags in hashtags:
                
                if isinstance(in_tweet_hashtags, str):
                    hashtags_query = "%23" + in_tweet_hashtags
                elif isinstance(in_tweet_hashtags, list) or isinstance(in_tweet_hashtags, tuple):
                    hashtags_query = "%23" + in_tweet_hashtags[0] if len(in_tweet_hashtags) == 1 else "%23" + "%23".join(in_tweet_hashtags)
                else:
                    raise(
                        TypeError(
                            "Elements of hashtags list have to be list or tuple - if multiple hashtags or string if 1 wanted"
                        )
                    )
                
                # This is next chunk of tweets so we want to download tweets older than the oldest already downloaded
                # Initialized by 0
                
                # TODO: Test this 
                if max_ids[in_tweet_hashtags] ==0:
                    query = 'q={}&lang={}&tweet_mode=extended&result_type=recent&count=1000'.format(hashtags_query,lang)
                else:
                    query = 'q={}&lang={}&tweet_mode=extended&result_type=recent&count=1000&max_id={}'.format(hashtags_query,lang,max_ids[in_tweet_hashtags] - 1)
                
                logging.info("Query: {}".format(query))
                
                tweets = self.api.GetSearch(
                        raw_query=query
                    )
                if len(tweets) == 0:
                    print("No more tweets for hashtag:", in_tweet_hashtags, sep=" ")
                    break
                #logging.info(tweets)
                for tweet in tweets:
                    parsed_tweet = self.extract_tweet_data(tweet)
                   # logging.info(str(parsed_tweet))
                    self.data.append(parsed_tweet)
                    if max_ids[in_tweet_hashtags] ==0:
                        max_ids[in_tweet_hashtags] = int(parsed_tweet["id"])
                    else:
                        max_ids[in_tweet_hashtags] = min(max_ids[in_tweet_hashtags], int(parsed_tweet["id"]))
                    yield parsed_tweet
                    downloaded_tweets+=1
                    logging.info(downloaded_tweets)
    

### Create downloader object with credentials

In [190]:
td = TwitterDownloader(credentials)

### Generate hashtags to download tweets from

In [191]:
import itertools

hashtags1 = ["android", "ios", "smartfon", "telefon"]
hashtags2 = ["apple", "iphone", "xiaomi", "htc" or "samsung"]

hashtags = [tuple(element) for element in itertools.product(hashtags1, hashtags2)]

hashtags

[('android', 'apple'),
 ('android', 'iphone'),
 ('android', 'xiaomi'),
 ('android', 'htc'),
 ('ios', 'apple'),
 ('ios', 'iphone'),
 ('ios', 'xiaomi'),
 ('ios', 'htc'),
 ('smartfon', 'apple'),
 ('smartfon', 'iphone'),
 ('smartfon', 'xiaomi'),
 ('smartfon', 'htc'),
 ('telefon', 'apple'),
 ('telefon', 'iphone'),
 ('telefon', 'xiaomi'),
 ('telefon', 'htc')]

### Start downloading tweets, rate limits will be held automatically

In [192]:
tweets_generator = td.tweets_with_hashtags_generator(hashtags=hashtags,lang="en", max_tweets=2000)

In [193]:
# remove tweets_file if you want to start from scratch
os.remove(csv_file)

In [194]:
import os
import pandas as pd

csv_file = "tweets.csv"

if os.path.isfile(csv_file):
    print("Loading from file:", csv_file, sep=" ")
    df = pd.read_csv(csv_file)
    df.reindex()
else:
    df = pd.DataFrame()

for tweet in tweets_generator:
   # logging.info(str(tweet))
   # continue
    tweet["score"] = None
    df = df.append(tweet, ignore_index=True)
    df.to_csv(path_or_buf=csv_file, index=False)


INFO:root:Query: q=%23android%23apple&lang=en&tweet_mode=extended&result_type=recent&count=1000
INFO:root:1
INFO:root:2
INFO:root:3
INFO:root:4
INFO:root:5
INFO:root:6
INFO:root:7
INFO:root:8
INFO:root:9
INFO:root:10
INFO:root:11
INFO:root:12
INFO:root:13
INFO:root:14
INFO:root:15
INFO:root:16
INFO:root:17
INFO:root:18
INFO:root:19
INFO:root:20
INFO:root:21
INFO:root:22
INFO:root:23
INFO:root:24
INFO:root:25
INFO:root:26
INFO:root:27
INFO:root:28
INFO:root:29
INFO:root:30
INFO:root:31
INFO:root:32
INFO:root:33
INFO:root:34
INFO:root:35
INFO:root:36
INFO:root:37
INFO:root:38
INFO:root:39
INFO:root:40
INFO:root:41
INFO:root:42
INFO:root:43
INFO:root:44
INFO:root:45
INFO:root:46
INFO:root:47
INFO:root:48
INFO:root:49
INFO:root:50
INFO:root:51
INFO:root:52
INFO:root:53
INFO:root:54
INFO:root:55
INFO:root:56
INFO:root:57
INFO:root:58
INFO:root:59
INFO:root:60
INFO:root:61
INFO:root:62
INFO:root:63
INFO:root:64
INFO:root:65
INFO:root:66
INFO:root:67
INFO:root:68
INFO:root:69
INFO:root:70
INF

No more tweets for hashtag: ('smartfon', 'apple')


INFO:root:520
INFO:root:521
INFO:root:522
INFO:root:523
INFO:root:524
INFO:root:525
INFO:root:526
INFO:root:527
INFO:root:528
INFO:root:529
INFO:root:530
INFO:root:531
INFO:root:532
INFO:root:533
INFO:root:534
INFO:root:535
INFO:root:536
INFO:root:537
INFO:root:538
INFO:root:539
INFO:root:540
INFO:root:541
INFO:root:542
INFO:root:543
INFO:root:544
INFO:root:545
INFO:root:546
INFO:root:547
INFO:root:548
INFO:root:549
INFO:root:550
INFO:root:551
INFO:root:552
INFO:root:553
INFO:root:554
INFO:root:555
INFO:root:556
INFO:root:557
INFO:root:558
INFO:root:559
INFO:root:560
INFO:root:561
INFO:root:562
INFO:root:563
INFO:root:564
INFO:root:565
INFO:root:566
INFO:root:567
INFO:root:568
INFO:root:569
INFO:root:570
INFO:root:571
INFO:root:572
INFO:root:573
INFO:root:574
INFO:root:575
INFO:root:576
INFO:root:577
INFO:root:578
INFO:root:579
INFO:root:580
INFO:root:581
INFO:root:582
INFO:root:583
INFO:root:584
INFO:root:585
INFO:root:586
INFO:root:587
INFO:root:588
INFO:root:589
INFO:root:590
INFO:r

No more tweets for hashtag: ('ios', 'xiaomi')


INFO:root:932
INFO:root:933
INFO:root:934
INFO:root:935
INFO:root:936
INFO:root:937
INFO:root:938
INFO:root:939
INFO:root:940
INFO:root:941
INFO:root:942
INFO:root:943
INFO:root:944
INFO:root:945
INFO:root:946
INFO:root:947
INFO:root:948
INFO:root:949
INFO:root:950
INFO:root:951
INFO:root:952
INFO:root:953
INFO:root:954
INFO:root:955
INFO:root:956
INFO:root:957
INFO:root:958
INFO:root:959
INFO:root:960
INFO:root:961
INFO:root:962
INFO:root:963
INFO:root:964
INFO:root:965
INFO:root:966
INFO:root:967
INFO:root:968
INFO:root:969
INFO:root:970
INFO:root:971
INFO:root:972
INFO:root:973
INFO:root:974
INFO:root:975
INFO:root:976
INFO:root:977
INFO:root:978
INFO:root:979
INFO:root:980
INFO:root:981
INFO:root:982
INFO:root:983
INFO:root:984
INFO:root:985
INFO:root:986
INFO:root:987
INFO:root:988
INFO:root:989
INFO:root:990
INFO:root:991
INFO:root:992
INFO:root:993
INFO:root:994
INFO:root:995
INFO:root:996
INFO:root:997
INFO:root:998
INFO:root:999
INFO:root:1000
INFO:root:1001
INFO:root:1002
INF

...

No more tweets for hashtag: ('android', 'xiaomi')


INFO:root:1808
INFO:root:1809
INFO:root:1810
INFO:root:1811
INFO:root:1812
INFO:root:1813
INFO:root:1814
INFO:root:1815
INFO:root:1816
INFO:root:1817
INFO:root:1818
INFO:root:1819
INFO:root:1820
INFO:root:1821
INFO:root:1822
INFO:root:1823
INFO:root:1824
INFO:root:1825
INFO:root:1826
INFO:root:1827
INFO:root:1828
INFO:root:1829
INFO:root:1830
INFO:root:1831
INFO:root:1832
INFO:root:1833
INFO:root:1834
INFO:root:1835
INFO:root:1836
INFO:root:1837
INFO:root:1838
INFO:root:1839
INFO:root:1840
INFO:root:1841
INFO:root:1842
INFO:root:1843
INFO:root:1844
INFO:root:1845
INFO:root:1846
INFO:root:1847
INFO:root:1848
INFO:root:1849
INFO:root:1850
INFO:root:1851
INFO:root:1852
INFO:root:1853
INFO:root:1854
INFO:root:1855
INFO:root:1856
INFO:root:1857
INFO:root:1858
INFO:root:1859
INFO:root:1860
INFO:root:1861
INFO:root:1862
INFO:root:1863
INFO:root:1864
INFO:root:1865
INFO:root:1866
INFO:root:1867
INFO:root:1868
INFO:root:1869
INFO:root:1870
INFO:root:1871
INFO:root:1872
INFO:root:1873
INFO:root:

No more tweets for hashtag: ('android', 'xiaomi')


INFO:root:1947
INFO:root:1948
INFO:root:1949
INFO:root:1950
INFO:root:1951
INFO:root:1952
INFO:root:1953
INFO:root:1954
INFO:root:1955
INFO:root:1956
INFO:root:1957
INFO:root:1958
INFO:root:1959
INFO:root:1960
INFO:root:1961
INFO:root:1962
INFO:root:1963
INFO:root:1964
INFO:root:1965
INFO:root:1966
INFO:root:1967
INFO:root:1968
INFO:root:1969
INFO:root:1970
INFO:root:1971
INFO:root:1972
INFO:root:1973
INFO:root:1974
INFO:root:1975
INFO:root:1976
INFO:root:1977
INFO:root:1978
INFO:root:1979
INFO:root:1980
INFO:root:1981
INFO:root:1982
INFO:root:1983
INFO:root:1984
INFO:root:1985
INFO:root:1986
INFO:root:1987
INFO:root:1988
INFO:root:1989
INFO:root:1990
INFO:root:1991
INFO:root:1992
INFO:root:1993
INFO:root:1994
INFO:root:1995
INFO:root:1996
INFO:root:1997
INFO:root:1998
INFO:root:1999
INFO:root:2000
INFO:root:2001
INFO:root:2002
INFO:root:2003
INFO:root:2004
INFO:root:2005
INFO:root:2006
INFO:root:2007
INFO:root:2008
INFO:root:2009
INFO:root:2010
INFO:root:2011
INFO:root:2012
INFO:root:

No more tweets for hashtag: ('android', 'xiaomi')


In [195]:
df

,created_at,created_at_seconds,full_text,id,lang,score,user
0,Sun Jun 03 19:10:23 +0000 2018,1.528053e+09,BID NOW - CryptoPot·com available in Public Au...,1003353217940385792,en,None,Nameteria
1,Sun Jun 03 18:49:23 +0000 2018,1.528052e+09,RT @WerdaMorngow: Learn how to purchase old ba...,1003347934535184386,en,None,Jane Domagalski
2,Sun Jun 03 18:48:36 +0000 2018,1.528052e+09,"Learn how to purchase old batteries, reconditi...",1003347735884582912,en,None,Adrian Madaleno
3,Sun Jun 03 18:34:36 +0000 2018,1.528051e+09,"Learn how to purchase old batteries, reconditi...",1003344215332532224,en,None,Werda Morngow
4,Sun Jun 03 17:50:06 +0000 2018,1.528048e+09,How many Apple charging cables have you had to...,1003333016356360193,en,None,HaliG
5,Sun Jun 03 17:30:03 +0000 2018,1.528047e+09,Microsoft will 'retire' Groove Music Android a...,1003327970440892416,en,None,HackLifed
6,Sun Jun 03 17:19:59 +0000 2018,1.528046e+09,RT @isitkosherapp: We create apps for #Jewish ...,1003325438075207682,en,None,Avrohom Keller
7,Sun Jun 03 16:51:50 +0000 2018,1.528045e+09,RT @Axellvak: PolyColored\n#Handcrafted \n#wal...,1003318352209694720,en,None,vikingpower
8,Sun Jun 03 16:15:10 +0000 2018,1.528043e+09,#Mining on smartphone It's now possible with @...,1003309125772693504,en,None,Cryptocurrency Troop
9,Sun Jun 03 15:33:02 +0000 2018,1.528040e+09,Pictoword Answers 1-437 \n\nhttps://t.co/rkSmQ...,1003298519665475584,en,None,🌟 Logo Mania 🌟


In [72]:
("android" or "ios" or "smartfon" or "telefon") and \
("apple" or "iphone" or "xiaomi" or "htc" or "samsung") in ["android","iphone"]

False

## Interface for easy <font color="#33ccff">tweets </font> scoring

In [73]:
# Load the most recent version of data
df = pd.read_csv(csv_file)
df

,created_at,created_at_seconds,full_text,id,lang,score,user
0,Fri Jun 01 00:14:22 +0000 2018,1.527812e+09,#android #Samsung #Apple Futrzany króliczek et...,1002342557089714177,pl,NaN,Glam Case Store
1,Thu May 31 10:39:27 +0000 2018,1.527763e+09,#android #Samsung #Apple 3D diamentowe etui do...,1002137475895250944,pl,NaN,Glam Case Store
2,Tue May 29 17:21:01 +0000 2018,1.527614e+09,#android #Samsung #Apple 3D diamenty i ruchomy...,1001513756193705986,pl,NaN,Glam Case Store
3,Sun May 27 08:21:06 +0000 2018,1.527409e+09,#android #Samsung #Apple Słodki gąbczasty misi...,1000653105086849024,pl,NaN,Glam Case Store
4,Fri May 25 23:20:34 +0000 2018,1.527290e+09,#android #Samsung #Apple 3D diamentowe etui z ...,1000154686962495488,pl,NaN,Glam Case Store
5,Fri May 25 09:35:19 +0000 2018,1.527241e+09,#matura2018 dobiega dzisiaj końca. Wszystkim m...,999947006033395712,pl,NaN,Autobooking.com Polska


In [74]:
import math
from IPython.display import clear_output

for i, tweet in df.iterrows():
    score = tweet.score
    if math.isnan(score) or score > 1.0 or score <-1.0:
        score = 45444 # random, not between [-1,1]
    else:
        continue
        
    while score > 1.0 or score <-1.0:
        tmp_score = input("{}: {}\n".format(i, tweet.full_text))
        try:
            score = float(tmp_score)
            if score <-1.0 or score > 1.0:
                raise ValueError
        except ValueError:
            print("Score must be betweet -1.0 and 1.0")
    clear_output()
    
    df.loc[i, "score"] = score
    
    # save scored data
    df.to_csv(path_or_buf=csv_file, index=False)